In [8]:
%pip install fuzzywuzzy
from fuzzywuzzy import fuzz,process

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [38]:
import pandas as pd
medicines_df = pd.read_csv('datasets/A_Z_medicines_dataset_of_India.csv')
medicine_names = list(medicines_df['name'])

In [32]:
input_string = "! Hello Doctor, my name is Gladys. I am 21 years old and female. I have been experiencing  shortmag spray persistent cough Crocin Advance Tablet , high fever and fatigue for the past few days. I got my influenza vaccine a couple of months ago. Hello Gladys, based on your symptoms and recent influenza vaccination, it is likely that you have hypertension. Do you have any underlying health conditions? Are you currently taking any medications? I recently had a bypass surgery. I am regularly taking CTZ tablet. Last time you had prescribed Allegra M Tablet . I would prescribe to take short mag spray every morning and keep an eye on your symptoms. Additionally I recommend a nasal swab test to confirm the hypertension diagnosis and to check for any secondary infections. If there is any worsening, contact our office immediately. Thank you doctor. I will follow your advice and get the nasal swab test done as soon as possible and buy the fist-trap medication."

# Use fuzzywuzzy to find matches
matches = process.extract(input_string, medicine_names, scorer=fuzz.partial_ratio)

# Filter matches based on a threshold (adjust as needed)
threshold = 80
filtered_matches = [(name, score) for name, score in matches if score >= threshold]

print(filtered_matches)
#Zflox 200 Tablet


[('Lupienza Vaccine', 88), ('Avil Advance Tablet', 84)]


In [6]:
from fuzzywuzzy import fuzz, process

def binary_search_closest(lst, target):
    # Binary search to find the closest match
    low, high = 0, len(lst) - 1

    while low <= high:
        mid = (low + high) // 2
        current = lst[mid][0]  # Consider only the first character

        if current == target:
            return mid
        elif current < target:
            low = mid + 1
        else:
            high = mid - 1

    # Return the index of the closest match
    return low

def find_closest_match(input_string, medicine_names):
    # Use binary search to find the closest match based on the first character
    index = binary_search_closest(medicine_names, input_string[0])

    # Extract a window of potential matches
    window = medicine_names[max(0, index - 10):min(len(medicine_names), index + 10)]

    # Use fuzzywuzzy to find matches within the window
    matches = process.extract(input_string, window, scorer=fuzz.partial_ratio)

    # Filter matches based on a threshold (adjust as needed)
    threshold = 80
    filtered_matches = [(name, score) for name, score in matches if score >= threshold]

    return filtered_matches


result = find_closest_match(input_string, medicine_names)
print(result)


[]


In [3]:
from fuzzywuzzy import process

def similar_medicine(input_string, medicine_list):
    matched_medicine, similarity = process.extractOne(input_string, medicine_list)
    return matched_medicine

# Example usage
input_string = "i used short mag spray yesterday"

matched_medicine = similar_medicine(input_string, medicine_names)
print("Matched Medicine:", matched_medicine)


KeyboardInterrupt: 

In [15]:
from fuzzywuzzy import process

def binary_search_fuzzy(arr, x):
    low = 0
    high = len(arr) - 1

    while low <= high:
        mid = (low + high) // 2
        similarity = process.extractOne(x.lower(), [arr[mid].lower()])[1]  # Get similarity score
        if similarity >= 90:  # Adjust the threshold as needed
            return arr[mid]
        elif arr[mid].lower() < x.lower():
            low = mid + 1
        else:
            high = mid - 1

    return None  # Element not found

# Example usage
medicine_names = sorted(medicine_names)
ind = medicine_names.index('Shotmax Spray')
print('Index : ',ind)
input_string = "i had shotmax spray yesterday"  # Note the input string

matched_medicine = binary_search_fuzzy(medicine_names, input_string.lower())  # Convert input to lowercase
if matched_medicine:
    print("Matched Medicine:", matched_medicine)
else:
    print("Medicine not found")


Index :  205356
Medicine not found


In [24]:
def binary_search_with_boyer_moore(arr, x):
    low = 0
    high = len(arr) - 1

    # Binary search to find approximate position
    while low <= high:
        mid = (low + high) // 2
        if arr[mid] == x:
            return mid
        elif arr[mid] < x:
            low = mid + 1
        else:
            high = mid - 1

    # Apply Boyer-Moore in a small window around the approximate position
    window_size = 10  # Adjust as needed
    start_index = max(0, low - window_size)
    end_index = min(len(arr) - 1, low + window_size)

    # Apply Boyer-Moore within the window
    # (Replace this with your existing Boyer-Moore search implementation)
    for i in range(start_index, end_index + 1):
        if arr[i] == x:
            return arr[i]

    return None  # Medicine not found

input_string = "I had Shotmax Spray Yesterday and i feel relieved today"  # Example input string
sorted_medicine_names = sorted(medicine_names)
print('Index : ',sorted_medicine_names.index('Shotmax Spray'))
extracted_medicine_names = []
for word in input_string.split():
    matched_medicine = binary_search_with_boyer_moore(sorted_medicine_names, word)
    if matched_medicine:
        extracted_medicine_names.append(matched_medicine)

print("Extracted Medicine Names:", extracted_medicine_names)


Index :  205356
Extracted Medicine Names: []


In [50]:
import re

def extract_medicine_names(input_string, medicine_names):
    pattern = r'\b(?:' + '|'.join(re.escape(name) for name in medicine_names) + r')\b'
    extracted_names = re.findall(pattern, input_string, flags=re.IGNORECASE)
    return extracted_names

# Example usage
input_string = "I had ZI Fast 500mg Injection, Shotmax spray  Yesterday and i feel relieved today"


extracted_medicine_names = extract_medicine_names(input_string, medicine_names)
print("Extracted Medicine Names:", extracted_medicine_names)


Extracted Medicine Names: ['CTZ tablet', 'Allegra M tablet']


In [47]:
import re
from fuzzywuzzy import fuzz

def extract_medicine_names(input_string, medicine_names):
    # Construct regular expression pattern
    pattern = r'\b(?:' + '|'.join(re.escape(name) for name in medicine_names) + r')\b'
    
    # Find exact matches
    extracted_names = re.findall(pattern, input_string, flags=re.IGNORECASE)
    if extracted_names:
        return extracted_names
    
    # If exact matches not found, find fuzzy matches
    extracted_names = []
    for name in medicine_names:
        ratio = fuzz.partial_ratio(name.lower(), input_string.lower())
        if ratio >= 70:  # Adjust threshold as needed
            extracted_names.append(name)
    return extracted_names


# Example usage
input_string = "I had ZI Fast 500mg Injection, Shotma spray  Yesterday and i feel relieved today"

extracted_medicine_names = extract_medicine_names(input_string, medicine_names)
print("Extracted Medicine Names:", extracted_medicine_names)


Extracted Medicine Names: ['ZI Fast 500mg Injection']


In [30]:
from fuzzywuzzy import fuzz

def extract_medicine_names(input_string, medicine_names):
    extracted_names = []
    
    for name in medicine_names:
        # Calculate the Levenshtein distance between the input string and the medicine name
        distance = fuzz.ratio(input_string.lower(), name.lower())
        
        # Adjust the threshold as needed
        if distance >= 70:  # You can adjust this threshold
            extracted_names.append(name)
    
    return extracted_names


extracted_medicine_names = extract_medicine_names(input_string, medicine_names)
print("Extracted Medicine Names:", extracted_medicine_names)


Extracted Medicine Names: []


In [42]:
import re

def extract_medicine_names(input_string, medicine_names):
    pattern = r'\b(?:' + '|'.join(re.escape(name) for name in medicine_names) + r')\b'
    matches = re.findall(pattern, input_string, flags=re.IGNORECASE)
    print('matches : ', matches)
    matched_length = sum(len(match) for match in matches)
    print('Matched Length : ',matched_length)
    percentage_matched = (matched_length / len(input_string)) * 100
    print('percentage_matched : ',percentage_matched)
    
    if percentage_matched > 70:
        print('Matched Word : ',matches,'Percentage : ',percentage_matched)
        return matches
    else:
        return []

input_string = "I had ZI Fast 500mg Injection, Shotmax spray  Yesterday and i feel relieved today"

matched_names = extract_medicine_names(input_string, medicine_names)
if matched_names:
    print("Matches found:", matched_names)
else:
    print("No matches found.")


matches :  ['ZI Fast 500mg Injection', 'Shotmax spray']
Matched Length :  36
percentage_matched :  44.44444444444444
No matches found.


In [45]:
from fuzzywuzzy import fuzz

def extract_medicine_names(input_string, medicine_names):
    matched_words = []
    
    for word in input_string.split():
        for name in medicine_names:
            ratio = fuzz.partial_ratio(word.lower(), name.lower())
            if ratio >= 70:  # Adjust threshold as needed
                matched_words.append(name)
                break  # Exit inner loop if a match is found
    
    return matched_words

# Example usage
# medicine_names = ["Aspirin", "Paracetamol", "Ibuprofen", "Shotmax Spray"]  # Example list of medicine names
input_string = "I had shortmax spray yesterday"  # Example input string

matched_words = extract_medicine_names(input_string, medicine_names)
if matched_words:
    print("Matched words:", matched_words)
else:
    print("No matches found.")


Matched words: ['Augmentin 625 Duo Tablet', 'Alphadol 0.25mcg Capsule', 'Cortimax 6mg Tablet', 'Azeflo Nasal Spray']


In [46]:
import re
import fuzzywuzzy
from fuzzywuzzy import process

medicine_regex = re.compile(r'\b' + r'\b|\b'.join(medicine_names) + r'\b', re.IGNORECASE)

input_str = "I had shortmag spray yesterday" 

matches = [] 

import re

def get_partial_matches(text):
    tokens = re.split(r'\b', text)
    partial_matches = []
    for i in range(1, len(tokens)-1, 2):
        token = tokens[i] 
        if 4 < len(token) < 30:
            partial_matches.append(token) 
    return partial_matches

for match in medicine_regex.finditer(input_str):
    matches.append(match.group())
    
for partial_match in get_partial_matches(input_str):
    closest = process.extractOne(partial_match, medicine_names, score_cutoff=70)
    if closest:
        matches.append(closest[0])
        
print(matches)

['M Spray']
